In [33]:
import numpy as np
import scipy.stats as sts
from scipy.linalg import sqrtm

GIG = sts.geninvgauss 

def sample_gh(n, mu, gamma, A, p, b):
    """  
    Generalized hyperbolic (multivariate) sampler using sto representation with GIG mixture
    mu = location
    gamma = skewness
    A = sqrtm of the scatter matrix 
    p, b relative to the shape of dist (p negative and small b for heavy tails)
    """
    assert A.shape[0] == mu.shape[0]
    W = GIG(p,b).rvs(size=n) # always a scalar
    Z = sts.multivariate_normal.rvs(mean=np.zeros_like(mu),cov=np.eye(mu.shape[0]),size=n)
    X = mu + W[:,None]*gamma + np.sqrt(W)[:,None]*(Z @ A.T)
    return X

def einsum_moment(k):
    letters = [chr(ord('i')+t) for t in range(k)]
    left = ','.join(['n'+L for L in letters])
    right = ''.join(letters)
    return f"{left}->{right}"

def comoment(X,k):
    X = np.asarray(X)
    n = X.shape[0]
    Xc = X - X.mean(axis=0, keepdims=True)
    operation = einsum_moment(k)
    args = [Xc]*k
    return np.einsum(operation,*args,optimize=True) / n


In [31]:
k = 4
scale = sts.wishart.rvs(df=k+2, scale = np.eye(k))
A = sqrtm(scale)
mu = sts.norm.rvs(0,2,size=k) # locs
gamma = sts.norm.rvs(-0.5,1,size=k) #skewness
p = -0.5 
b = 0.5 

In [34]:
N = 10_000_000
X = sample_gh(N,mu,gamma,A,p,b)

In [35]:
mean = np.mean(X,axis=0)
cov = np.cov(X.T)
coskew = comoment(X,3)
cokurt = comoment(X,4)

In [46]:
coskew.round(1)

array([[[-205.3,  -42.5,  -67.8,  -45.3],
        [ -42.5,  -28. ,  -13.6,  -24.5],
        [ -67.8,  -13.6,  -36. ,   -5.6],
        [ -45.3,  -24.5,   -5.6,  -45.6]],

       [[ -42.5,  -28. ,  -13.6,  -24.5],
        [ -28. ,  -10.5,   -7.3,  -10.5],
        [ -13.6,   -7.3,   -6.3,   -5.2],
        [ -24.5,  -10.5,   -5.2,  -13.7]],

       [[ -67.8,  -13.6,  -36. ,   -5.6],
        [ -13.6,   -7.3,   -6.3,   -5.2],
        [ -36. ,   -6.3,  -17. ,   -3.2],
        [  -5.6,   -5.2,   -3.2,   -7.9]],

       [[ -45.3,  -24.5,   -5.6,  -45.6],
        [ -24.5,  -10.5,   -5.2,  -13.7],
        [  -5.6,   -5.2,   -3.2,   -7.9],
        [ -45.6,  -13.7,   -7.9,  -24.4]]])

In [53]:
cokurt.round(-1)

array([[[[5730., 1190., 1880., 1260.],
         [1190.,  600.,  380.,  540.],
         [1880.,  380.,  870.,  240.],
         [1260.,  540.,  240.,  940.]],

        [[1190.,  600.,  380.,  540.],
         [ 600.,  260.,  190.,  240.],
         [ 380.,  190.,  170.,  140.],
         [ 540.,  240.,  140.,  310.]],

        [[1880.,  380.,  870.,  240.],
         [ 380.,  190.,  170.,  140.],
         [ 870.,  170.,  440.,   90.],
         [ 240.,  140.,   90.,  220.]],

        [[1260.,  540.,  240.,  940.],
         [ 540.,  240.,  140.,  310.],
         [ 240.,  140.,   90.,  220.],
         [ 940.,  310.,  220.,  500.]]],


       [[[1190.,  600.,  380.,  540.],
         [ 600.,  260.,  190.,  240.],
         [ 380.,  190.,  170.,  140.],
         [ 540.,  240.,  140.,  310.]],

        [[ 600.,  260.,  190.,  240.],
         [ 260.,  240.,   80.,  200.],
         [ 190.,   80.,  110.,   40.],
         [ 240.,  200.,   40.,  250.]],

        [[ 380.,  190.,  170.,  140.],
         [ 